In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# 1. 貼上剛才定義的 Class
# ==========================================
class OptionCompositeAnalyzer:
    def __init__(self, short_leg_row, long_leg_row):
        # 模擬從 DataFrame Row 讀取資料
        self.short = short_leg_row
        self.long = long_leg_row
        
        self.short_k = float(self.short['履約價'])
        self.long_k = float(self.long['履約價'])
        
        self.short_price = float(self.short['收盤價'])
        self.long_price = float(self.long['收盤價'])
        
        # 計算 Net Credit
        self.max_profit = self.short_price - self.long_price
        
        # 計算 Max Loss (價差寬度 - Net Credit)
        self.width = abs(self.long_k - self.short_k)
        self.max_loss = self.width - self.max_profit
        
        # 讀取勝率 (Win Rate = 1 - Itm_Prob)
        self.prob_itm_short = float(self.short['Itm_Prob'])
        self.win_rate = 1.0 - self.prob_itm_short

    def calculate_kelly(self, fraction_multiplier=1.0):
        # 防呆與極端值處理
        if self.max_loss <= 0:
            return {"kelly_suggestion": 0.0, "reason": "Arbitrage or Data Error (No Risk)"}
        
        if self.max_profit <= 0:
            return {"kelly_suggestion": 0.0, "reason": "No Profit"}

        # b = 賠率 (獲利 / 虧損)
        b = self.max_profit / self.max_loss
        
        # p = 勝率
        p = self.win_rate
        q = 1.0 - p
        
        # 凱利公式: f = p - q/b
        # 若 b 極小 (虧損風險極大)，f 會變負值
        try:
            f = p - (q / b)
        except ZeroDivisionError:
            f = -1.0
        
        f_adjusted = f * fraction_multiplier
        
        return {
            "win_rate": round(p, 4),
            "odds_b": round(b, 4),           # 賺賠比
            "kelly_full": round(f, 4),       # 原始凱利
            "kelly_suggestion": max(0.0, round(f_adjusted, 4)), # 負值歸零
            "is_tradable": f > 0,
            "details": f"Win:{p:.1%} / Risk-Reward:1:{b:.2f}"
        }

# ==========================================
# 2. 測試案例生成 (Test Harness)
# ==========================================
def create_mock_row(strike, price, itm_prob):
    """ 快速建立模擬的 Pandas Series，格式模仿您的 df_opt """
    data = {
        '履約價': strike,
        '收盤價': price,
        'Itm_Prob': itm_prob,
        '買賣權': 'Call', # 為了情境完整性
        '到期月份(週別)': '202206'
    }
    return pd.Series(data)

def run_test_scenario(name, short_k, short_p, short_prob, long_k, long_p):
    print(f"--- 測試情境: {name} ---")
    
    # 建立模擬資料
    # Short Leg: 賣 18000 Call, 價格 short_p, 被穿價機率 short_prob
    s_row = create_mock_row(short_k, short_p, short_prob)
    # Long Leg: 買 18200 Call (保護), 價格 long_p
    l_row = create_mock_row(long_k, long_p, 0.0) # Long leg 的 prob 在此策略邏輯較不重要
    
    analyzer = OptionCompositeAnalyzer(s_row, l_row)
    
    # 測試 Full Kelly 與 Half Kelly
    res_full = analyzer.calculate_kelly(fraction_multiplier=1.0)
    res_half = analyzer.calculate_kelly(fraction_multiplier=0.5)
    
    print(f"部位: Sell {short_k}@{short_p} / Buy {long_k}@{long_p}")
    print(f"最大獲利: {analyzer.max_profit:.1f}, 最大虧損: {analyzer.max_loss:.1f}")
    print(f"勝率 (1-Itm): {res_full['win_rate']:.1%} (假設)")
    print(f"賠率 (b): {res_full['odds_b']}")
    print(f"原始凱利值 (f*): {res_full['kelly_full']}")
    print(f"建議倉位 (Half Kelly): {res_half['kelly_suggestion']:.2%}  <-- 這是您要的答案")
    print(f"可否交易: {res_full['is_tradable']}")
    print("\n")

# ==========================================
# 3. 執行測試
# ==========================================

# 情境 A: 典型的優質 OTM 價差單
# 寬度 200, 收 40 點, 風險 160 點 (賠率 0.25)
# 但勝率很高 90% (Itm_Prob = 0.1)
# 期望值 E = 0.9*40 - 0.1*160 = 36 - 16 = +20 (正期望值)
run_test_scenario(
    name="A. 高勝率穩健單 (Good Setup)",
    short_k=18000, short_p=50, short_prob=0.1,  # 90% 勝率
    long_k=18200, long_p=10
)

# 情境 B: 期望值為負的垃圾單
# 寬度 200, 收 40 點, 風險 160 點
# 但勝率只有 70% (Itm_Prob = 0.3)
# 期望值 E = 0.7*40 - 0.3*160 = 28 - 48 = -20 (負期望值)
# 凱利公式應該要建議倉位為 0
run_test_scenario(
    name="B. 負期望值單 (Avoid This)",
    short_k=18000, short_p=50, short_prob=0.3,  # 70% 勝率
    long_k=18200, long_p=10
)

# 情境 C: 高風險高報酬 (接近價平)
# 寬度 200, 收 90 點, 風險 110 點 (賠率 ~0.81)
# 勝率 60% (Itm_Prob = 0.4)
# 期望值 E = 0.6*90 - 0.4*110 = 54 - 44 = +10 (微幅正期望值)
run_test_scenario(
    name="C. 積極型交易 (Aggressive)",
    short_k=18000, short_p=100, short_prob=0.4, # 60% 勝率
    long_k=18200, long_p=10
)

--- 測試情境: A. 高勝率穩健單 (Good Setup) ---
部位: Sell 18000@50 / Buy 18200@10
最大獲利: 40.0, 最大虧損: 160.0
勝率 (1-Itm): 90.0% (假設)
賠率 (b): 0.25
原始凱利值 (f*): 0.5
建議倉位 (Half Kelly): 25.00%  <-- 這是您要的答案
可否交易: True


--- 測試情境: B. 負期望值單 (Avoid This) ---
部位: Sell 18000@50 / Buy 18200@10
最大獲利: 40.0, 最大虧損: 160.0
勝率 (1-Itm): 70.0% (假設)
賠率 (b): 0.25
原始凱利值 (f*): -0.5
建議倉位 (Half Kelly): 0.00%  <-- 這是您要的答案
可否交易: False


--- 測試情境: C. 積極型交易 (Aggressive) ---
部位: Sell 18000@100 / Buy 18200@10
最大獲利: 90.0, 最大虧損: 110.0
勝率 (1-Itm): 60.0% (假設)
賠率 (b): 0.8182
原始凱利值 (f*): 0.1111
建議倉位 (Half Kelly): 5.56%  <-- 這是您要的答案
可否交易: True




In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad

# ==========================================
# 1. 核心分析類別 (The Analyzer)
# ==========================================
class OptionCompositeAnalyzer:
    def __init__(self, short_leg_row, long_leg_row, risk_free_rate=0.01):
        """
        初始化分析器
        :param short_leg_row: 賣方部位資料 (pd.Series)
        :param long_leg_row: 買方部位資料 (pd.Series)
        :param risk_free_rate: 無風險利率
        """
        self.r = risk_free_rate
        self.short = short_leg_row
        self.long = long_leg_row

        # 提取基礎參數
        self.k_short = float(self.short['履約價'])
        self.k_long = float(self.long['履約價'])
        
        self.p_short = float(self.short['收盤價'])
        self.p_long = float(self.long['收盤價'])
        
        # 提取 Greeks 與機率參數
        self.iv = float(self.short['Implied_Volatility'])
        self.T = float(self.short['dT'])
        self.prob_itm_short = float(self.short['Itm_Prob']) # N(d2)
        
        # 為了積分需要，嘗試取得標的價格 S
        # 假設資料流中已經包含了 'S' (期貨現價)，若無則用 BS反推或報錯
        if 'S' in self.short:
            self.S_ref = float(self.short['S'])
        else:
            # Fallback: 這裡僅為防呆，實際回測應確保有傳入 S
            self.S_ref = self.k_short # 暫時替代

        # 基礎損益結構
        self.net_credit = self.p_short - self.p_long
        self.width = abs(self.k_long - self.k_short)
        self.max_loss = self.width - self.net_credit

    # ------------------------------------------------
    # 方案 A: 二元簡化模型 (Binary Model)
    # ------------------------------------------------
    def calculate_binary_kelly(self, kelly_multiplier=1.0):
        """
        邏輯：只要結算價 > Short Strike，視為全輸 (Max Loss)。
        """
        # 勝率 (Win Rate): 1 - ITM機率
        win_rate = 1.0 - self.prob_itm_short
        
        # 賠率 (Odds b): 獲利 / 虧損
        if self.max_loss <= 0: return self._error_result("No Risk")
        b = self.net_credit / self.max_loss
        
        # 期望值 (EV)
        ev = (win_rate * self.net_credit) - ((1 - win_rate) * self.max_loss)
        
        # 凱利公式: f = p - q/b
        p = win_rate
        q = 1 - p
        try:
            f = p - (q / b)
        except ZeroDivisionError:
            f = 0
            
        return {
            "Method": "Binary (A)",
            "Win_Rate": win_rate,
            "EV": ev,
            "Kelly_Full": f,
            "Kelly_Sugg": max(0, f * kelly_multiplier),
            "Note": "忽略緩衝區，較悲觀"
        }

    # ------------------------------------------------
    # 方案 B: 積分精確模型 (Integration Model)
    # ------------------------------------------------
    def calculate_integral_kelly(self, kelly_multiplier=1.0):
        """
        邏輯：對 (損益函數 * 機率密度) 進行積分，考慮損益兩平點緩衝區。
        """
        sigma_t = self.iv * np.sqrt(self.T)
        if sigma_t == 0: return self._error_result("Zero Volatility")

        # 對數常態分佈參數
        mu_log = np.log(self.S_ref) + (self.r - 0.5 * self.iv**2) * self.T
        
        # 定義 PDF (機率密度函數)
        def lognormal_pdf(x):
            if x <= 0: return 0
            coeff = 1 / (x * sigma_t * np.sqrt(2 * np.pi))
            exponent = -((np.log(x) - mu_log)**2) / (2 * sigma_t**2)
            return coeff * np.exp(exponent)

        # 定義到期損益函數 (Payoff) - Bear Call Spread
        def payoff(x):
            # 賣權利金 - (Short虧損 - Long獲利)
            intrinsic_loss = max(0, x - self.k_short) - max(0, x - self.k_long)
            return self.net_credit - intrinsic_loss

        # 積分計算期望值 E[PnL]
        # 積分範圍：S +/- 4個標準差 (覆蓋 99.99%)
        lower = self.S_ref * np.exp(-4 * sigma_t)
        upper = self.S_ref * np.exp(4 * sigma_t)
        
        # 計算 EV (PnL * PDF)
        ev, _ = quad(lambda x: payoff(x) * lognormal_pdf(x), lower, upper)
        
        # 計算真實勝率 (POP): 積分 PnL > 0 的區間
        # 損益兩平點 (Breakeven) = K_short + Net_Credit
        be_point = self.k_short + self.net_credit
        # 只要 S < BE_Point 都是賺錢的 (包含賺少少)
        true_win_rate, _ = quad(lognormal_pdf, 0, be_point)

        # 連續型凱利近似: f = Edge / Odds = EV / Max_Loss
        if self.max_loss <= 0: 
            f = 0
        else:
            f = ev / self.max_loss

        return {
            "Method": "Integral (B)",
            "Win_Rate": true_win_rate,
            "EV": ev,
            "Kelly_Full": f,
            "Kelly_Sugg": max(0, f * kelly_multiplier),
            "Note": "精確計算，含緩衝區"
        }

    def _error_result(self, msg):
        return {"Method": "Error", "Note": msg, "Kelly_Sugg": 0, "EV": 0, "Win_Rate": 0}

# ==========================================
# 2. 模擬資料生成器 (Data Mocker)
# ==========================================
def create_mock_pair(S, K_short, K_long, T_days, IV):
    """
    快速生成符合 df_opt 格式的測試資料
    """
    r = 0.01
    T = T_days / 365.0
    
    # 簡單計算 BS Call Price & Delta (Itm_Prob) 用於填充資料
    def bs_call(s, k, t, v, r):
        d1 = (np.log(s/k) + (r + 0.5*v**2)*t) / (v*np.sqrt(t))
        d2 = d1 - v*np.sqrt(t)
        price = s * norm.cdf(d1) - k * np.exp(-r*t) * norm.cdf(d2)
        prob = norm.cdf(d2) # N(d2) is Itm Probability for Call
        return price, prob

    p_short, prob_short = bs_call(S, K_short, T, IV, r)
    p_long, _ = bs_call(S, K_long, T, IV, r)
    
    # 建立 Series
    row_short = pd.Series({
        '履約價': K_short, '收盤價': p_short, 'Itm_Prob': prob_short,
        'Implied_Volatility': IV, 'dT': T, 'S': S, '契約': 'TXO'
    })
    row_long = pd.Series({
        '履約價': K_long, '收盤價': p_long, 'Itm_Prob': 0, # Long方機率不影響計算
        'Implied_Volatility': IV, 'dT': T, 'S': S
    })
    
    return row_short, row_long

# ==========================================
# 3. 測試與比較腳本 (Comparison Test)
# ==========================================
def run_comparison_test():
    print(">> 開始執行方案 A vs 方案 B 對比測試...\n")
    
    scenarios = [
        # 情境 1: 深價外 (Deep OTM) - 兩者應該差不多
        {"name": "1. 安全牌 (Deep OTM)", "S": 17000, "K_s": 18000, "K_l": 18200, "T": 30, "IV": 0.2},
        
        # 情境 2: 邊緣單 (The Trap) - 關鍵測試點！
        # 股價離履約價很近，二元模型會因為穿價機率稍高而叫你放棄
        # 但積分模型會發現其實還有緩衝區 (損益兩平點)，期望值可能是正的
        {"name": "2. 邊緣機會 (Near OTM)", "S": 17800, "K_s": 18000, "K_l": 18200, "T": 15, "IV": 0.15},
        
        # 情境 3: 價內單 (ITM) - 應該都要賠錢
        {"name": "3. 必死單 (ITM)", "S": 18100, "K_s": 18000, "K_l": 18200, "T": 10, "IV": 0.2}
    ]
    
    results = []
    
    for sc in scenarios:
        # 生成資料
        r_short, r_long = create_mock_pair(sc['S'], sc['K_s'], sc['K_l'], sc['T'], sc['IV'])
        
        # 初始化分析器
        analyzer = OptionCompositeAnalyzer(r_short, r_long)
        
        # 執行兩種計算
        res_a = analyzer.calculate_binary_kelly(kelly_multiplier=0.5) # 用半凱利
        res_b = analyzer.calculate_integral_kelly(kelly_multiplier=0.5)
        
        # 整理結果
        results.append({
            "情境": sc['name'],
            "策略": "Sell Call Spread",
            "方案": "A (二元)",
            "勝率(%)": f"{res_a['Win_Rate']:.1%}",
            "期望值(EV)": f"{res_a['EV']:.1f}",
            "建議倉位": f"{res_a['Kelly_Sugg']:.2%}"
        })
        results.append({
            "情境": "",
            "策略": "",
            "方案": "B (積分)",
            "勝率(%)": f"{res_b['Win_Rate']:.1%}",
            "期望值(EV)": f"{res_b['EV']:.1f}",
            "建議倉位": f"{res_b['Kelly_Sugg']:.2%}"
        })
        results.append({}) # 空行

    # 輸出表格
    df_res = pd.DataFrame(results)
    # print(df_res.to_markdown(index=False))
    print(df_res.to_string(index=False))

# 執行
if __name__ == "__main__":
    run_comparison_test()

>> 開始執行方案 A vs 方案 B 對比測試...

                情境               策略     方案 勝率(%) 期望值(EV)  建議倉位
 1. 安全牌 (Deep OTM) Sell Call Spread A (二元) 84.4%    -4.3 0.00%
                                    B (積分) 85.0%    -0.0 0.00%
               NaN              NaN    NaN   NaN     NaN   NaN
2. 邊緣機會 (Near OTM) Sell Call Spread A (二元) 64.4%   -12.9 0.00%
                                    B (積分) 68.3%    -0.0 0.00%
               NaN              NaN    NaN   NaN     NaN   NaN
      3. 必死單 (ITM) Sell Call Spread A (二元) 43.7%   -13.3 0.00%
                                    B (積分) 50.3%    -0.0 0.00%
               NaN              NaN    NaN   NaN     NaN   NaN
